In [1]:
#Basic Libraries
import numpy as np
import pandas as pd

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Text Handling Libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# clustering
from sklearn.cluster import KMeans

#
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [2]:
# Function for removing NonAscii characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

# Function for converting into lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

# Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# recommendation
def get_recommendation(title, sim,products,indices):
    idx = indices[title]
    sim_scores = list(enumerate(sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    product_indices = [i[0] for i in sim_scores]
    return products.iloc[product_indices]

In [3]:
df = pd.read_csv('data/zue_all_materials.csv')
df['zzibrnd'] = df['zzibrnd'].astype(str)
df['TherapeuticClass'] = df['TherapeuticClass'].astype(str)
df['PrincipalName'] = df['PrincipalName'].astype(str)
df['Description'] = df['Description'].astype(str)
df['zzibrnd'] = df['Description'].astype(str)
df['GenericMaterialName'] = df['Description'].astype(str)
df.info()


C:\Users\rupak\AppData\Local\Temp\ipykernel_13084\2365512333.py:1: DtypeWarning: Columns (0,14,15,17,20,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/zue_all_materials.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220901 entries, 0 to 220900
Data columns (total 50 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   matnr                       220901 non-null  object 
 1   zzprncode                   220852 non-null  float64
 2   zzibrnd                     220901 non-null  object 
 3   mtart                       220901 non-null  object 
 4   matkl                       220898 non-null  object 
 5   TherapeuticClass            220901 non-null  object 
 6   PrincipalName               220901 non-null  object 
 7   mstae                       213974 non-null  object 
 8   mstde                       220901 non-null  int64  
 9   mstav                       153406 non-null  object 
 10  mstdv                       220901 non-null  int64  
 11  zzdwrx                      220901 non-null  object 
 12  spart                       220848 non-null  float64
 13  vtweg         

In [4]:
df['matnr'] = df['matnr'].drop_duplicates()
df

,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,mstav,...,taxm4,taxm5,taxm6,taxm7,taxm8,taxm9,taxim,tatyp,vtext,idnlf
0,26095737,102521.0,"QIAxcel, Basic Agreement,o PM",ZF06,ZMD,Non-IMS classified,QIAGEN SINGAPORE PTE LTD,LR,20460101,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,9241203
1,23265332,102544.0,DELTA C-TAPER HEAD 28mm -2.5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,18283
2,23265333,102544.0,DELTA C-TAPER HEAD 32mm +0,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183200
3,23265334,102544.0,DELTA C-TAPER HEAD 32mm +5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183205
4,23265335,102544.0,DELTA C-TAPER HEAD 32mm +2.5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220896,NaN,102017.0,人類上皮因子接受體第二蛋白陽性玻片,ZF03,ZMD,Non-IMS classified,台灣羅氏醫療診斷設備(股)公司,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,05273510001
220897,NaN,102017.0,*PATHWAT HER2/NEU4IN 1'Kit,ZF03,ZMD,Non-IMS classified,台灣羅氏醫療診斷設備(股)公司,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,05273510001
220898,NaN,102017.0,ＨＥＲ人類上皮細胞接受器用一級抗體,ZF03,ZMD,Non-IMS classified,台灣羅氏醫療診斷設備(股)公司,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,05278368001
220899,NaN,102017.0,Path. Anti-Her-2/NEU 50Test 1'PC,ZF03,ZMD,Non-IMS classified,台灣羅氏醫療診斷設備(股)公司,LR,99901231,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,05278368001


In [5]:
df_clean = df.reset_index()
brands = df_clean['zzibrnd']
indices = pd.Series(df_clean.index, index=df_clean['zzibrnd'])
df_clean.head()

,index,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,taxm4,taxm5,taxm6,taxm7,taxm8,taxm9,taxim,tatyp,vtext,idnlf
0,0,26095737,102521.0,"QIAxcel, Basic Agreement,o PM",ZF06,ZMD,Non-IMS classified,QIAGEN SINGAPORE PTE LTD,LR,20460101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,9241203
1,1,23265332,102544.0,DELTA C-TAPER HEAD 28mm -2.5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,18283
2,2,23265333,102544.0,DELTA C-TAPER HEAD 32mm +0,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183200
3,3,23265334,102544.0,DELTA C-TAPER HEAD 32mm +5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183205
4,4,23265335,102544.0,DELTA C-TAPER HEAD 32mm +2.5,ZF03,ZMD,Non-IMS classified,Stryker (Thailand) Ltd.,LR,99901231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MWST,Product : Full Tax,183225


In [12]:
df_clean['soup'] = df_clean['Description'] 
df_clean['soup'][0]

'QIAxcel, Basic Agreement,o PM'

In [13]:
# Applying all the functions in description and storing as a cleaned_desc
df_clean['soup'] = df_clean['soup'].apply(_removeNonAscii)
df_clean['soup'] = df_clean.soup.apply(func = make_lower_case)
df_clean['soup'] = df_clean.soup.apply(func=remove_punctuation)
df_clean['soup'] = df_clean.soup.apply(func=remove_html)
df_clean['soup'][0]

'qiaxcel basic agreement o pm'

In [14]:
df_clean[['soup','zzibrnd']]

,soup,zzibrnd
0,qiaxcel basic agreement o pm,"QIAxcel, Basic Agreement,o PM"
1,delta c taper head 28mm 2 5,DELTA C-TAPER HEAD 28mm -2.5
2,delta c taper head 32mm 0,DELTA C-TAPER HEAD 32mm +0
3,delta c taper head 32mm 5,DELTA C-TAPER HEAD 32mm +5
4,delta c taper head 32mm 2 5,DELTA C-TAPER HEAD 32mm +2.5
...,...,...
220896,,人類上皮因子接受體第二蛋白陽性玻片
220897,pathwat her2 neu4in 1 kit,*PATHWAT HER2/NEU4IN 1'Kit
220898,,ＨＥＲ人類上皮細胞接受器用一級抗體
220899,path anti her 2 neu 50test 1 pc,Path. Anti-Her-2/NEU 50Test 1'PC


In [15]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_clean['soup'])
tfidf_matrix.shape

(220901, 187466)

In [16]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

MemoryError: Unable to allocate 364. GiB for an array with shape (220901, 220901) and data type float64

In [53]:
searchstr = 'BEPANTHEN FIRST AIDCREAM30G FREE 3.5G'
idx = get_recommendation(searchstr,cosine_sim,brands,indices).values
idx

array(['MENTHOLATUM WATERLIPFRAGRANCE FREE 3.5G',
       'BEPANTHEN BABY BATHLIQUID200ML', 'BEPANTHEN FIRST AID CREAM 30G',
       'BEPANTHEN OINT 100GVALUEPACK RM25',
       'BEPANTHEN OINTMENT 30G TWIN PACK',
       'BEPANTHEN OINTMENT 30G VPRM10.50',
       'BEPANTHEN OINT 30G VALUE PACK RM10',
       'BEPANTHEN OINT 100GFREE 5X7ML BABY BATH', 'THERAPY LIPBALM3.5G',
       'LIPICE LEMON (NEW)3.5G'], dtype=object)

In [54]:
df_clean.iloc[['4782']]

,index,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,prat1,prat2,prat3,prat4,prat5,prat6,prat7,prat8,prat9,soup
4782,4782,23063718,101363.0,FOLTENE ANTI AGING SHAMPOO 200ML,ZF03,ZCS,Other dermatological preparations,PARVUS SDN BHD,LR,20250120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,foltene anti aging shampoo 200ml parvus sdn bh...


In [56]:
df_clean[df_clean['zzibrnd'].isin(idx)]

,index,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,...,prat1,prat2,prat3,prat4,prat5,prat6,prat7,prat8,prat9,soup
633,633,23064705,101369.0,BEPANTHEN OINT 100GFREE 5X7ML BABY BATH,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen oint 100gfree 5x7ml baby bath bayer ...
898,898,23064870,101369.0,BEPANTHEN FIRST AID CREAM 30G,ZF03,ZPO,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen first aid cream 30g bayer co malaysi...
1407,1407,23064727,101369.0,BEPANTHEN OINTMENT 30G VPRM10.50,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen ointment 30g vprm10 50 bayer co mala...
1683,1683,23064713,101369.0,BEPANTHEN OINT 100GVALUEPACK RM25,ZF03,ZCH,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen oint 100gvaluepack rm25 bayer co mal...
1752,1752,23064920,101369.0,BEPANTHEN BABY BATHLIQUID200ML,ZF03,ZCS,Wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen baby bathliquid200ml bayer co malays...
2406,2406,23064725,101369.0,BEPANTHEN OINTMENT 30G TWIN PACK,ZF03,ZPE,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen ointment 30g twin pack bayer co mala...
2564,2564,23071367,101242.0,LIPICE LEMON (NEW)3.5G,ZF03,ZPC,"Emollients, protectives",ROHTO-MENTHOLATUM (M) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lipice lemon new 3 5g rohto mentholatum m sdn ...
2822,2822,23071373,101242.0,MENTHOLATUM WATERLIPFRAGRANCE FREE 3.5G,ZF03,ZPC,"Emollients, protectives",ROHTO-MENTHOLATUM (M) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mentholatum waterlipfragrance free 3 5g rohto ...
3158,3158,23064712,101369.0,BEPANTHEN OINT 30G VALUE PACK RM10,ZF03,ZCH,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bepanthen oint 30g value pack rm10 bayer co ma...
4348,4348,23071016,101242.0,THERAPY LIPBALM3.5G,ZF03,ZPC,"Emollients, protectives",ROHTO-MENTHOLATUM (M) SDN BHD,LR,20250101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,therapy lipbalm3 5g rohto mentholatum m sdn bh...


In [59]:
searchstr = '(Z) JANUMET XR 50/1000 TABS 56\'S'
idx = get_recommendation(searchstr,cosine_sim,brands,indices).values
idx

array(["(Z) JANUMET XR 100/1000 TABS 28'S",
       "(Z) JANUMET HAARLEM TABS 50/1000MG 56'S",
       "JANUMET HAARLEMTABS50/500MG 56'S",
       "JANUMET HAARLEMTABS50/1000MG 56'S",
       "JANUMET HAARLEMTABS50/850MG 56'S", "JANUVIATABS100MG28'S",
       "GALVUS METTAB 50/850MG60'S", "GALVUS METTAB 50/1000MG60'S",
       "<GLUCOPHAGE XR TABS500MG4X15'S", "AVANDAMETTAB4MG/500MG 56'S"],
      dtype=object)

In [60]:
searchstr = 'BEBELAC INFANT FORMULA STEP 1400G'
idx = get_recommendation(searchstr,cosine_sim,brands,indices).values
idx

array(['BEBELAC INFANT FORMULA STEP 1800G',
       'DUPRO FOLLOW-UP FORMULA STEP 21KG',
       'DUPRO FOLLOW-UP FORMULA STEP 2650G',
       'NEOCATE LCP INFANT FORMULA 400G (MY)\xa0\xa0\xa0\xa0',
       'DUPRO FOLLOW-UP FORMULA STEP 21.6KG', 'MAMEX CHERISH 1400G',
       'BEBELAC FOLLOW-UP FORMULASTEP 2800G',
       'MAMIL STEP 3 1.7KG (MY) WHA\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
       'DULAC INFANT FORMULASTEP 11KG', 'DULAC INFANT FORMULASTEP 1350G'],
      dtype=object)

In [63]:
searchstr = 'DUREX PLAY SWEET STRAWBEERY 100ML'
idx = get_recommendation(searchstr,cosine_sim,brands,indices).values
idx

array(['DUREX PLAY WARMING 100ML', "DUREX PLAY 100ML 1'S/BOX",
       'DUREX PLAY SOOTHINGALOEVERA100ML', 'DUREX PLAY TINGLE100ML',
       "DUREX PLAY 50ML 1'S/BOX", "DUREX PLAY O 15ML 1'S/BOX",
       "DUREX PLAY VIBRATINGCONDOM 1'S/BOX",
       'DUREX PLAY MASSAGE2 IN 1200ML',
       "DUREX PLAY MASSAGE SENSUALYLANG1'S/BOX",
       'DUREX LOVE 12SFOC PLAY MSGSACHT'], dtype=object)

In [65]:
searchstr = 'PPD ESCITALOPRAMTAB10MG28\'S'
idx = get_recommendation(searchstr,cosine_sim,brands,indices).values
idx

array(["PPD QUETIAPINETAB25MG60'S", "PPD QUETIAPINETAB100MG60'S",
       'PPD BISOPROLOLTAB5MG100S', "PPD BISOPROLOLTAB2.5MG100'S", 'nan',
       'nan', 'nan', 'PPD AMOXI+CLAVU ACIDTAB 500MG+125MG 100S',
       "LISDENE UD MYTAB20MG30'S", 'nan'], dtype=object)